In [1]:
import boto3
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def get_embedding(prompt_data, boto3_bedrock):
    body = json.dumps({"inputText": prompt_data})
    modelId = "amazon.titan-embed-text-v1"  # (Change this to try different embedding models)
    accept = "application/json"
    contentType = "application/json"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    return embedding


def invoke_model(prompt_data, question, boto3_bedrock):

    model_input_dict = {'anthropic_version': 'bedrock-2023-05-31',
                        'max_tokens': 20000,
                        'system': prompt_data,
                        "messages" : question}

    body = json.dumps(model_input_dict)

    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'  # change this to use a different version from the model provider
    accept = "application/json"
    contentType = "application/json"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    response_string = response_body['content'][0]['text']
    response_string = response_string.strip()

    return response_string


def answer_question(question, boto3_bedrock, prompt=""):
    user_request = "<request>" + question + "</request>"
    messages = [{"role": "user", "content": user_request}]
    return invoke_model(prompt, messages, boto3_bedrock)


def compute_closest_texts(question, embeddings, boto3_bedrock, embedding_column = 'embedding'):
    target_embedding = get_embedding(question, boto3_bedrock)

    vector_array = np.array(target_embedding)
    matrix_array = np.array([emb for emb in embeddings[embedding_column]])

    vector_array = vector_array.reshape(1, -1)

    # Compute cosine similarity
    similarities = cosine_similarity(vector_array, matrix_array)

    n_texts = len(similarities)-1
    
    # Flatten the similarities matrix and get the indices of the top 3 similarities
    top_indices = np.argpartition(similarities.flatten(), -n_texts)[-n_texts:]
    top_indices = top_indices[np.argsort(similarities.flatten()[top_indices])][::-1]
    
    return top_indices


def enhance_prompt(question, df, boto3_bedrock):
    top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding', num_contexts=5)

    prompt = f"""
    <task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
    y tú debes responderla basado en los textos de contexto que están a continuación. Evita
    decir cosas que no se mencionen en los textos. Si los textos no contienen información relevante para
    responder la pregunta, responde que no tienes una respuesta. En tu respuesta evita mencionar los textos,
    con frases como "de acuerdo con los textos proporcionados". Tu respuesta debe ser escrita en un párrafo y evita
    escribirla como una lista de puntos.
    """

    for i in range(num_contexts):
        prompt += f""" 
        <context_text>
        {df.loc[top_indices[i], 'content']}
        </context_text>
        """

    return prompt

Inicializar el cliente a Bedrock:

In [2]:
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name="us-east-1"
)

In [20]:
model_input_dict = {'anthropic_version': 'bedrock-2023-05-31',
                    'max_tokens': 20000,
                    'system': '',
                    "messages" : [{"role": "user", "content": "Cuál es la capital de Colombia?"}]}

body = json.dumps(model_input_dict)

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

response_string = response_body['content'][0]['text']
response_string = response_string.strip()

print(response_string)

La capital de Colombia es Bogotá.

Bogotá es la ciudad más grande y poblada de Colombia, así como su capital política, económica, administrativa, industrial, artística y cultural. Está ubicada en el centro del país, en la región conocida como la sabana de Bogotá, formando parte de la región andina.

Algunas datos interesantes sobre Bogotá:

- Fue fundada el 6 de agosto de 1538 por Gonzalo Jiménez de Quesada.

- Tiene una población metropolitana de cerca de 10 millones de habitantes.

- Es una de las ciudades más altas del mundo, ubicada a unos 2.600 metros sobre el nivel del mar.

- Cuenta con una amplia oferta cultural, museos, sitios turísticos y patrimonio colonial.

- Es un importante centro financiero y de negocios en Latinoamérica.

- La Plaza de Bolívar y el barrio La Candelaria conforman su centro histórico colonial.

Así que, en resumen, Bogotá es la inconfundible capital de la República de Colombia y una de las ciudades más relevantes del país.


### Ejemplo sin prompt:

In [21]:
question='Cuál es el escándalo por los carrotanques?'

print(answer_question(question, boto3_bedrock))

El escándalo de los "carrotanques" se refiere a un caso de corrupción que salió a la luz en Colombia en 2009, relacionado con la contratación irregular de unos vehículos blindados por parte del gobierno colombiano.

Los hechos fueron los siguientes:

- En 2004, durante el gobierno de Álvaro Uribe, el Ministerio de Defensa firmó un contrato para adquirir unos vehículos militares blindados provenientes de Turquía, conocidos como "carrotanques".

- El contrato se hizo por un valor de cerca de $50 millones de dólares a pesar de que estos vehículos tenían un precio mucho menor en el mercado.

- Se descubrió que hubo irregularidades en el proceso de contratación, sobrecostos injustificados y presuntos sobornos por parte de intermediarios.

- Según las investigaciones, varias personas cercanas al expresidente Uribe y altos funcionarios del Ministerio de Defensa estuvieron involucrados en este escándalo de corrupción.

- Se abrieron procesos judiciales y algunas personas fueron condenadas por 

### Retrieval Augmented Generation (RAG):

In [5]:
df = pd.read_csv("noticias_politica_2024.csv")

In [6]:
df['embedding'] = ''
for i in range(len(df)):
    df.at[i, 'embedding'] = get_embedding(df.loc[i, 'content'], boto3_bedrock)

In [7]:
df

,content,date,headline,description,embedding
0,En el expediente contra la campaña Petro Presi...,2024-05-13T00:30:32.386Z,Expediente contra Petro: CNE afirma que se omi...,Estos son los hallazgos sobre el uso de una ae...,"[0.38358793, -0.30966458, 0.06684935, 0.389035..."
1,No paran los ataques del presidente de la Repú...,2024-05-12T23:12:48.839Z,Petro se pegó de Bernie Sanders para atacar a ...,El jefe de Estado en una publicación en su cue...,"[0.40859932, -0.1044974, 0.21654549, 0.2305278..."
2,SEMANA: Usted hizo parte del Consejo Superior ...,2024-05-13T15:03:00.019Z,“Recomiendo a Petro que vaya al psicólogo porq...,"El reconocido psicólogo Humberto Rosanía, quie...","[0.3065858, -0.31919742, 0.28361088, 0.5296659..."
3,No guardó silencio el presidente de la Repúbli...,2024-05-12T20:07:33.567Z,Petro responde con risa a acusaciones de Anton...,El jefe de Estado se pronunció en sus redes so...,"[0.3289917, -0.30707383, 0.2519821, 0.49302283..."
4,"Este domingo 12 de mayo, el alcalde de Bogotá,...",2024-05-12T18:29:39.661Z,Alcalde Carlos Fernando Galán anuncia su prime...,El mandatario de Bogotá busca financiación par...,"[0.20697084, -0.49794617, 0.022642154, -0.0563..."
...,...,...,...,...,...
835,"De manera sorpresiva, el presidente de la Repú...",2024-04-10T16:11:45.297Z,Presidente Petro reveló que en Caracas sostuvo...,"El martes (9 de abril), el jefe de Estado tuvo...","[0.60042536, -0.42830598, 0.35029408, 0.451171..."
836,"El alcalde de Bogotá, Carlos Fernando Galán, c...",2024-04-10T15:32:24.729Z,Atención: alcalde Carlos Fernando Galán anunci...,El mandatario aseguró que hubo operativos en t...,"[0.7651423, -0.1471936, -0.027458835, -0.02127..."
837,El expresidente Álvaro Uribe se refirió a la d...,2024-04-10T15:28:41.947Z,Álvaro Uribe asegura que su llamado a juicio e...,El exmandatario se refirió a la decisión judic...,"[0.5359693, -0.2784308, 0.16754454, 0.20510463..."
838,El sistema de salud está en crisis. Esa es una...,2024-04-10T15:17:32.108Z,Grave alerta de las EPS por “complejo” panoram...,"Ana María Vesga, presidenta de Acemi, el gremi...","[0.35191458, -0.26534426, 0.22674501, 0.496737..."


In [22]:
df.loc[0, 'content']

'En el expediente contra la campaña Petro Presidente de 2022 reposan pruebas sobre la presunta omisión del reporte de las horas de vuelo que se contrataron con la compañía Sociedad Aérea de Ibagué S.A.S., conocida también como Sadi.La cuestión de la avioneta que llevó a Gustavo Petro en algunas de sus correrías por todo el país venía siendo objeto de debate de la opinión pública desde hace varios meses, pero ahora el Consejo Nacional Electoral (CNE) lo tuvo en cuenta para abrir la investigación contra el actual mandatario.“Se observa la presunta manipulación de la información en relación con la generación de pagos por servicios de transporte aéreo realizados por la empresa Sadi toda vez que, de acuerdo a la información allegada por la Dian, se logra evidenciar la generación sistemática de facturas electrónicas que posteriormente son objeto de anulación, generando en consecuencia, una nueva facturación que modifica el valor y el concepto inicial”, se lee en el documento presentado por l

In [8]:
top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding')

In [9]:
df.loc[top_indices[0], 'content']

'Un escándalo de corrupción estalló en Colombia el 3 de mayo, luego de que Sneyder Pinilla, quien fuera subdirector de manejo del riesgo en la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD), testificara sobre las investigaciones que se adelantan en el caso de los carrotanque para la Guajira. Mientras avanzan las indagaciones, salió a relucir un video en el que el hoy presidente Gustavo Petro, promocionaba la candidatura de Olmedo López, exdirector de la UNGRD, que salió salpicado en el escándalo.En el video promocional, el hoy mandatario de los colombianos recomendaba a los ciudadanos que votaran “por gente decente”, en alusión al partido por el cual se lanzó al Senado Olmedo López. “Este 11 de marzo, no vote por los corruptos al Congreso de la República”, decía Petro en la pieza audiovisual, en invitaba a depositar el voto “en Senado y Cámara, por Decentes”.‘Nace para quitarle $50 billones a la corrupción’Luego del espaldarazo que le daba Petro al partido Decentes, aparecí

In [23]:
df.loc[top_indices[1], 'content']

'El director de la Unidad Nacional de Gestión del Riesgo, Carlos Carrillo, reconoció una vez más que la entidad que dirige fue cooptada por la corrupción. Lo hizo este lunes 6 de mayo desde la localidad de Ciudad Bolívar, en Bogotá.“Y no en este gobierno, esto viene desde que se creó la entidad. Esa clase política no se fue el 7 de agosto de 2022, a esa clase política corrupta todavía nos sigue correspondiendo sacarla del Gobierno. Sacarla“, dijo. El público que asistía al auditorio aplaudió.Carrillo manifestó: “Se colaron unos y traicionaron al señor presidente Gustavo Petro y traicionaron el proyecto de cambio. Esas personas no caben en este gobierno”.Los corruptos, añadió, “que se vayan a la derecha, al centro, pero en la izquierda no tienen espacio. No lo pueden tener. Eso es lo que estamos haciendo. Yo quisiera ver que otro gobierno hubiera sido capaz de coger la UNGRD, que es tan poderosa, y abrir la caja de pandora. Ninguno”.Carrillo intenta desmarcar al gobierno de Gustavo Petr

In [24]:
df.loc[top_indices[2], 'content']

'Luis Fernando Velasco descartó que tenga que ver algo con el escándalo de los carrotanques parqueados en La Guajira. Le respondió al exdirector Olmedo López quien sugirió en una entrevista que el alto funcionario del Gobierno estaría relacionado.“Es el típico caso del raterito que lo cogen con las manos en la masa y trata de generar una confusión para que la gente intente ver hacia otro lado. No ha habido señalamientos porque, si hay señalamientos, yo reaccionaré jurídicamente. Todo son rumores”, detalló el ministro Velasco.Velasco aclaró que no tiene nada que ver con el escándalo. “No firmé un solo contrato o convenio. Atendí la emergencia del Nevado del Ruiz con un convenio que tenía con las fuerzas militares”, dijo.El ministro del Interior llegó acompañado al búnker de la Fiscalía junto a su abogado defensor Ernesto Velasco Chaves, junto a quien presentó una demanda por injuria y calumnia en contra de Silvio Ramírez, un ciudadano que lo relacionó con el escándalo de los carrotanque

In [26]:
top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding')

prompt = f"""
<task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
y tú debes responderla basado en los textos de contexto que están a continuación. Evita
decir cosas que no se mencionen en los textos. Tu respuesta debe ser escrita en un párrafo y evita
escribirla como una lista de puntos.
"""

num_contexts = 5

for i in range(num_contexts):
    prompt += f""" 
    <context_text>
    {df.loc[top_indices[i], 'content']}
    </context_text>
    """

In [27]:
print(prompt)


<task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
y tú debes responderla basado en los textos de contexto que están a continuación. Evita
decir cosas que no se mencionen en los textos. Tu respuesta debe ser escrita en un párrafo y evita
escribirla como una lista de puntos.
 
    <context_text>
    Un escándalo de corrupción estalló en Colombia el 3 de mayo, luego de que Sneyder Pinilla, quien fuera subdirector de manejo del riesgo en la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD), testificara sobre las investigaciones que se adelantan en el caso de los carrotanque para la Guajira. Mientras avanzan las indagaciones, salió a relucir un video en el que el hoy presidente Gustavo Petro, promocionaba la candidatura de Olmedo López, exdirector de la UNGRD, que salió salpicado en el escándalo.En el video promocional, el hoy mandatario de los colombianos recomendaba a los ciudadanos que votaran “por gente decente”, en alusión al pa

In [28]:
question='Cuál es el escándalo por los carrotanques?'

print(answer_question(question, boto3_bedrock, prompt))

El escándalo de los carrotanques se refiere a un caso de presunta corrupción dentro de la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD) de Colombia. Los hechos son los siguientes:

La UNGRD adquirió 40 carrotanques con sobrecostos para llevar agua a la región de La Guajira, que sufre de escasez hídrica. Sin embargo, estos vehículos nunca funcionaron adecuadamente y terminaron parqueados sin prestar un servicio efectivo. El exsubdirector de la UNGRD, Sneyder Pinilla, denunció que parte del dinero desviado por los sobrecostos en la compra de los carrotanques habría sido utilizado para sobornar con millonarias sumas al presidente del Senado, Iván Name, y al presidente de la Cámara de Representantes, Andrés Calle, a cambio de que aprobaran las reformas impulsadas por el gobierno de Gustavo Petro.

Este escándalo ha salpicado al gobierno Petro, pues los altos funcionarios involucrados fueron nombrados por el actual mandatario. Además, ha generado indignación en La Guajira al no

In [14]:
question='Qué ha pasado con Nicolás Petro?'
prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

Nicolás Petro, hijo del presidente Gustavo Petro, se encuentra en el ojo del huracán luego de haber confesado ante las autoridades que hubo irregularidades en la financiación de la campaña presidencial de su padre en 2022. Específicamente, Nicolás Petro reconoció lo siguiente:

Admitió haber recibido dineros de personas cuestionables como el narcotraficante condenado Samuel Santander Lopesierra (alias 'El Hombre Marlboro') y el empresario Euclides Torres, para financiar de forma irregular la campaña de su padre. Estos recursos no fueron reportados debidamente.

Reveló que su padre estuvo al tanto de los aportes recibidos de Euclides Torres, a través del senador Pedro Flórez, para eventos de la campaña como tarimas, publicidad, transporte, etc. Aunque aseguró que Petro no sabía los detalles de en qué se gastaba ese dinero.

Confesó haberse apropiado de una suma millonaria que fue donada para la campaña presidencial, utilizando parte de esos recursos en la logística de la misma.

Por est

In [15]:
question='Hazme un resumen sobre el estado de la reforma a la salud'

prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

El proyecto de reforma a la salud del gobierno de Gustavo Petro ha atravesado por un proceso accidentado. La primera propuesta, impulsada por la exministra Carolina Corcho, fue archivada en el Congreso debido a la fuerte oposición de diversos sectores como pacientes, médicos, academia y expertos en el tema. Posteriormente, el gobierno logró concertar un nuevo texto con algunas EPS, pero este ha sido cuestionado por varios senadores que lideraron el hundimiento de la reforma inicial.

Los principales puntos de crítica al nuevo proyecto son que mantiene errores conceptuales de la propuesta original, no resuelve los desafíos estructurales del sistema de salud, carece de estudios técnicos sólidos y podría poner en riesgo el derecho fundamental a la salud. Además, se cuestiona que no haya sido concertado con todos los actores involucrados.

A pesar de las críticas, el gobierno tiene la intención de radicar este nuevo proyecto con mensaje de urgencia para que sea aprobado antes de finalizar 

In [16]:
question='Cuál fue la propuesta de una asamblea constituyente del presidente?'

prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

El presidente Gustavo Petro ha propuesto convocar una asamblea nacional constituyente que sea convocada directamente por el pueblo colombiano, no por él mismo como presidente. La idea es que el pueblo decida sobre los grandes cambios y reformas que Petro considera necesarios para el país, y que el Congreso no ha logrado aprobar. 

Petro ha insistido en que no busca cambiar toda la Constitución del 91, sino establecer un "acuerdo nacional" sobre temas fundamentales como la implementación del acuerdo de paz, las reformas agraria y política, garantizar derechos básicos como salud y pensiones, el ordenamiento territorial, entre otros puntos. Este acuerdo nacional sería llevado luego a un referendo constitucional para que el pueblo lo apruebe o rechace.

En resumen, Petro plantea una asamblea constituyente de origen popular para concretar reformas profundas, sin cambiar formalmente la Constitución vigente, aunque sí modificándola en aspectos clave a través de un referendo aprobado por los c

In [17]:
question='Cuál fue el escándalo por el manual tarifario del ministerio de salud?'

prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

Lamentablemente, en los textos proporcionados no se menciona nada sobre un escándalo relacionado con un manual tarifario del Ministerio de Salud. Los textos hablan sobre temas como la reforma a la salud propuesta por el gobierno de Gustavo Petro, el nuevo sistema de salud para los maestros administrado por Fiduprevisora, denuncias de corrupción en las EPS, deudas con proveedores de las subredes de salud en Bogotá, entre otros temas relacionados con el sistema de salud colombiano. Sin embargo, no se aborda ningún escándalo específico sobre un manual tarifario del Ministerio de Salud.
